Word frequencies already calculated

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
lyrics = pd.read_csv("./lyrics.csv")
songs = pd.read_csv("./songs.csv")

In [ ]:
lyrics.head()

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0


In [ ]:
songs.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMKI128F931D80D,006,SOSDCFG12AB0184647,Lena 20 År,ARSB5591187B99A848,fba3e876-68f1-4a1f-99d9-c604480202ba,Lena Philipsson,262.26893,0.529819,0.410229,1998,6010886,-1,0
1,TRMMMUT128F42646E8,(Looking For) The Heart Of Saturday,SOBARPM12A8C133DFF,Cover Girl,ARDW5AW1187FB55708,42222090-c5e5-4243-8582-c29bc8b63ec6,Shawn Colvin,216.47628,0.685503,0.446733,1994,3156269,-1,0
2,TRMMMQY128F92F0EA3,Ethos of Coercion,SOKOVRQ12A8C142811,Descend Into Depravity,ARGWPP11187B9AEF43,f76167bb-c117-4022-8b6b-54c796edf5c9,Dying Fetus,196.02240,0.734471,0.511976,2009,6782293,-1,0
3,TRMMMTK128F424EF7C,Rock-N-Rule,SOIMMJJ12AF72AD643,I'm Only A Man (Bonus Track Version),ARDT9VH1187B999C0B,6b22de04-fb48-44aa-bd02-c1427f635477,Emery,217.57342,0.738996,0.563367,2007,1501464,-1,0
4,TRMMMQV12903CA201E,I Made It Over,SOOUESZ12AB0189AFD,Let's Celebrate (He Is Risen),ARVF2AD1187FB47580,fd711779-5524-4ed3-8d68-da0b867caa34,Rev. Timothy Wright,353.77587,0.377190,0.290242,0,6345139,-1,0


Use lyrics file to calculate tf-idf (and have UI switch between that and word freq on command for per-song)

In [ ]:
#get distinct words, already in stem form
distinct_words = pd.unique(lyrics['word'])
print(distinct_words.shape)
print(distinct_words)

(5000,)
['i' 'the' 'you' ... 'pudiera' 'vuela' 'bla']


In [ ]:
#calculate number of documents
num_docs = pd.unique(lyrics["track_id"]).size
print(num_docs)

237662


In [ ]:
#get document frequency for words
lyrics["document_frequency"] = 1#default value
doc_freq_dict = lyrics.word.value_counts()
# it = 0
# for word in distinct_words:
#   if it % 100 == 0:
#     print("Iteration", it)
#   it += 1
#   # lyrics_subset_word = lyrics[lyrics['word'] == word]
#   # lyrics_subset_word.head()
#   word_doc_freq = lyrics.value_counts#number of documents containing word
#   # print(word_doc_freq)
#   doc_freq_dict[word] = word_doc_freq
#   # lyrics.loc[lyrics['word'] == word, "document_frequency"] = word_doc_freq
lyrics["document_frequency"] = lyrics["word"].map(doc_freq_dict)
lyrics.head()

,track_id,mxm_tid,word,count,is_test,document_frequency
0,TRAAAAV128F421A322,4623710,i,6,0,174095
1,TRAAAAV128F421A322,4623710,the,4,0,188401
2,TRAAAAV128F421A322,4623710,you,2,0,166772
3,TRAAAAV128F421A322,4623710,to,2,0,179264
4,TRAAAAV128F421A322,4623710,and,5,0,175917


In [ ]:
#construct tf-idf file analagous to lyrics.csv
tfidf = lyrics.copy()
tfidf.loc[:, "tf-idf"] = tfidf["count"] * np.log( num_docs / tfidf["document_frequency"])
tfidf.head()

,track_id,mxm_tid,word,count,is_test,document_frequency,tf-idf
0,TRAAAAV128F421A322,4623710,i,6,0,174095,1.867490
1,TRAAAAV128F421A322,4623710,the,4,0,188401,0.929107
2,TRAAAAV128F421A322,4623710,you,2,0,166772,0.708444
3,TRAAAAV128F421A322,4623710,to,2,0,179264,0.563980
4,TRAAAAV128F421A322,4623710,and,5,0,175917,1.504186


In [ ]:
tfidf_export = tfidf[["track_id", "mxm_tid", "word", "tf-idf", "is_test"]]
tfidf_export.head()
tfidf_export.to_csv("./tfidf.csv")
#same format as lyrics.csv, just with tf-idf instead of count

Option 1: use a sentiment analysis library, track sentiment dist changes per artist over time

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
my_sentiment_analyzer = SentimentIntensityAnalyzer()
#has built in sentiment analysis, designed for strings/sentences but will be using it for words

In [ ]:
my_sentiment_analyzer.polarity_scores("happy")

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719}

In [ ]:
#compound metric in [-1, 1] representing positivity
sent_dict = my_sentiment_analyzer.polarity_scores("happy")
sentiment_metric = sent_dict['pos'] - sent_dict['neg']
print(sentiment_metric)

1.0


In [ ]:
lyrics_with_sentiment = tfidf.copy()
lyrics_with_sentiment.head()

,track_id,mxm_tid,word,count,is_test,document_frequency,tf-idf
0,TRAAAAV128F421A322,4623710,i,6,0,174095,1.867490
1,TRAAAAV128F421A322,4623710,the,4,0,188401,0.929107
2,TRAAAAV128F421A322,4623710,you,2,0,166772,0.708444
3,TRAAAAV128F421A322,4623710,to,2,0,179264,0.563980
4,TRAAAAV128F421A322,4623710,and,5,0,175917,1.504186


In [ ]:
#get the sentiment analysis for the words
lyrics_with_sentiment["sentiment_pos"] = 0
lyrics_with_sentiment["sentiment_neu"] = 0
lyrics_with_sentiment["sentiment_neg"] = 0
lyrics_with_sentiment["summary_sentiment_metric"] = 0
#default values
#doing it by constructing dicts from words to these concepts, so pandas can hopefully make it faster
pos_dict = {}
neg_dict = {}
neu_dict = {}
summary_sentiment_dict = {}
it = 0
for word in distinct_words:
  if it % 100 == 0:
    print("Iteration", it)
  it += 1
  sentiment_dict = my_sentiment_analyzer.polarity_scores(word)
  sentiment_metric = sentiment_dict['pos'] - sentiment_dict['neg']
  # lyrics_with_sentiment.loc[lyrics_with_sentiment['word'] == word, ["sentiment_pos", "sentiment_neu", "sentiment_neg", "summary_sentiment_metric"]] = [sentiment_dict['pos'], sentiment_dict['neu'], sentiment_dict['neg'], sentiment_metric]
  pos_dict[word] = sentiment_dict["pos"]
  neg_dict[word] = sentiment_dict["neg"]
  neu_dict[word] = sentiment_dict["neu"]
  summary_sentiment_dict[word] = sentiment_metric
  
# lyrics_with_sentiment.head()

Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Iteration 1000
Iteration 1100
Iteration 1200
Iteration 1300
Iteration 1400
Iteration 1500
Iteration 1600
Iteration 1700
Iteration 1800
Iteration 1900
Iteration 2000
Iteration 2100
Iteration 2200
Iteration 2300
Iteration 2400
Iteration 2500
Iteration 2600
Iteration 2700
Iteration 2800
Iteration 2900
Iteration 3000
Iteration 3100
Iteration 3200
Iteration 3300
Iteration 3400
Iteration 3500
Iteration 3600
Iteration 3700
Iteration 3800
Iteration 3900
Iteration 4000
Iteration 4100
Iteration 4200
Iteration 4300
Iteration 4400
Iteration 4500
Iteration 4600
Iteration 4700
Iteration 4800
Iteration 4900


In [ ]:
lyrics_with_sentiment["sentiment_pos"] = lyrics_with_sentiment["word"].map(pos_dict)
lyrics_with_sentiment["sentiment_neu"] = lyrics_with_sentiment["word"].map(neu_dict)
lyrics_with_sentiment["sentiment_neg"] = lyrics_with_sentiment["word"].map(neg_dict)
lyrics_with_sentiment["summary_sentiment_metric"] = lyrics_with_sentiment["word"].map(summary_sentiment_dict)
lyrics_with_sentiment.head()

,track_id,mxm_tid,word,count,is_test,document_frequency,tf-idf,sentiment_pos,sentiment_neu,sentiment_neg,summary_sentiment_metric
0,TRAAAAV128F421A322,4623710,i,6,0,174095,1.867490,0.0,0.0,0.0,0.0
1,TRAAAAV128F421A322,4623710,the,4,0,188401,0.929107,0.0,1.0,0.0,0.0
2,TRAAAAV128F421A322,4623710,you,2,0,166772,0.708444,0.0,1.0,0.0,0.0
3,TRAAAAV128F421A322,4623710,to,2,0,179264,0.563980,0.0,1.0,0.0,0.0
4,TRAAAAV128F421A322,4623710,and,5,0,175917,1.504186,0.0,1.0,0.0,0.0


In [ ]:
songs.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMKI128F931D80D,006,SOSDCFG12AB0184647,Lena 20 År,ARSB5591187B99A848,fba3e876-68f1-4a1f-99d9-c604480202ba,Lena Philipsson,262.26893,0.529819,0.410229,1998,6010886,-1,0
1,TRMMMUT128F42646E8,(Looking For) The Heart Of Saturday,SOBARPM12A8C133DFF,Cover Girl,ARDW5AW1187FB55708,42222090-c5e5-4243-8582-c29bc8b63ec6,Shawn Colvin,216.47628,0.685503,0.446733,1994,3156269,-1,0
2,TRMMMQY128F92F0EA3,Ethos of Coercion,SOKOVRQ12A8C142811,Descend Into Depravity,ARGWPP11187B9AEF43,f76167bb-c117-4022-8b6b-54c796edf5c9,Dying Fetus,196.02240,0.734471,0.511976,2009,6782293,-1,0
3,TRMMMTK128F424EF7C,Rock-N-Rule,SOIMMJJ12AF72AD643,I'm Only A Man (Bonus Track Version),ARDT9VH1187B999C0B,6b22de04-fb48-44aa-bd02-c1427f635477,Emery,217.57342,0.738996,0.563367,2007,1501464,-1,0
4,TRMMMQV12903CA201E,I Made It Over,SOOUESZ12AB0189AFD,Let's Celebrate (He Is Risen),ARVF2AD1187FB47580,fd711779-5524-4ed3-8d68-da0b867caa34,Rev. Timothy Wright,353.77587,0.377190,0.290242,0,6345139,-1,0


In [ ]:
# songs["track_id"] = songs["track_id"].astype(str)
songs_compact = songs[["track_id", "year", "artist_id", "artist_name"]]
songs_compact.info()
#remove data with year as 0
songs_compact = songs_compact[songs_compact["year"] != 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237662 entries, 0 to 237661
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   track_id     237662 non-null  object
 1   year         237662 non-null  int64 
 2   artist_id    237662 non-null  object
 3   artist_name  237662 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.3+ MB


In [ ]:
lyrics_with_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19045332 entries, 0 to 19045331
Data columns (total 11 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   track_id                  object 
 1   mxm_tid                   int64  
 2   word                      object 
 3   count                     int64  
 4   is_test                   int64  
 5   document_frequency        int64  
 6   tf-idf                    float64
 7   sentiment_pos             float64
 8   sentiment_neu             float64
 9   sentiment_neg             float64
 10  summary_sentiment_metric  float64
dtypes: float64(5), int64(4), object(2)
memory usage: 1.6+ GB


In [ ]:
#do inner join to get artist info
# songs.head()
# songs["track_id"] = songs["track_id"].astype(str)
# lyrics_with_sentiment["track_id"] = lyrics_with_sentiment["track_id"].astype(str)
joined_dataset = songs_compact.merge(lyrics_with_sentiment, on="track_id", how="inner")
joined_dataset.head()

,track_id,year,artist_id,artist_name,mxm_tid,word,count,is_test,document_frequency,tf-idf,sentiment_pos,sentiment_neu,sentiment_neg,summary_sentiment_metric
0,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,i,23,1,174095,7.158712,0.0,0.0,0.0,0.0
1,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,the,5,1,188401,1.161384,0.0,1.0,0.0,0.0
2,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,you,13,1,166772,4.604885,0.0,1.0,0.0,0.0
3,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,to,10,1,179264,2.819899,0.0,1.0,0.0,0.0
4,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,and,4,1,175917,1.203349,0.0,1.0,0.0,0.0


In [ ]:
#multiply each sentiment category by the count, and then sum up over each artist for each year
joined_dataset["positive"] = joined_dataset["sentiment_pos"] * joined_dataset["count"]
joined_dataset["neutral"] = joined_dataset["sentiment_neu"] * joined_dataset["count"]
joined_dataset["negative"] = joined_dataset["sentiment_neg"] * joined_dataset["count"]
joined_dataset.head()

,track_id,year,artist_id,artist_name,mxm_tid,word,count,is_test,document_frequency,tf-idf,sentiment_pos,sentiment_neu,sentiment_neg,summary_sentiment_metric,positive,neutral,negative
0,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,i,23,1,174095,7.158712,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,the,5,1,188401,1.161384,0.0,1.0,0.0,0.0,0.0,5.0,0.0
2,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,you,13,1,166772,4.604885,0.0,1.0,0.0,0.0,0.0,13.0,0.0
3,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,to,10,1,179264,2.819899,0.0,1.0,0.0,0.0,0.0,10.0,0.0
4,TRMMMKI128F931D80D,1998,ARSB5591187B99A848,Lena Philipsson,4280619,and,4,1,175917,1.203349,0.0,1.0,0.0,0.0,0.0,4.0,0.0


In [ ]:
#group by artist_id and year, sum the counted vars
aggregate_sentiment = joined_dataset.groupby(['artist_id', 'artist_name', 'year']).agg({'positive':'sum','negative':'sum', 'neutral':'sum'})
aggregate_sentiment.head()

positive  negative  neutral
artist_id          artist_name      year                             
AR002UA1187B9A637D The Bristols     2001      26.0       9.0    246.0
AR00B1I1187FB433EB Eagle Eye Cherry 2000      12.0      25.0    453.0
                   Eagle-Eye Cherry 1996      20.0       9.0    234.0
                                    1997     131.0     104.0   2329.0
                                    1998      42.0      15.0    397.0

In [ ]:
aggregate_sentiment.head(30)

positive  negative  \
artist_id          artist_name                year                       
AR002UA1187B9A637D The Bristols               2001      26.0       9.0   
AR00B1I1187FB433EB Eagle Eye Cherry           2000      12.0      25.0   
                   Eagle-Eye Cherry           1996      20.0       9.0   
                                              1997     131.0     104.0   
                                              1998      42.0      15.0   
                                              2000     144.0      88.0   
                                              2001      21.0      21.0   
                                              2003      72.0      49.0   
                   Eagle-Eye Cherry / Santana 2000      26.0       7.0   
AR00FOZ1187FB5C9F3 autoKratz                  2008       0.0       0.0   
AR00JIO1187B9A5A15 Saigon                     2004     126.0     150.0   
                                              2007      38.0      54.0   
AR00MBZ1187B9B5DB1 Memphis Minnie             1930       3.0       3.0   
                                              1941      18.0       9.0   
                                              1991      60.0      41.0   
                                              1995       8.0       3.0   
                                              1997      14.0      40.0   
                                              1999      14.0      15.0   
                                              2000      20.0      30.0   
AR00RBU1187B9B3F1F Matthew Jay                2000       6.0       9.0   
                                              2001      54.0      28.0   
AR00TGQ1187B994F29 Paula Toller               2000      37.0       7.0   
                                              2007      10.0      12.0   
                                              2008       0.0       2.0   
AR00Y9I1187B999412 Akercocke                  1999      41.0      24.0   
                                              2001      20.0      21.0   
                                              2002      13.0      19.0   
                                              2003      49.0      52.0   
                                              2005      38.0      28.0   
                                              2007      35.0      26.0   

                                                    neutral  
artist_id          artist_name                year           
AR002UA1187B9A637D The Bristols               2001    246.0  
AR00B1I1187FB433EB Eagle Eye Cherry           2000    453.0  
                   Eagle-Eye Cherry           1996    234.0  
                                              1997   2329.0  
                                              1998    397.0  
                                              2000   1817.0  
                                              2001    376.0  
                                              2003   1548.0  
                   Eagle-Eye Cherry / Santana 2000    250.0  
AR00FOZ1187FB5C9F3 autoKratz                  2008     46.0  
AR00JIO1187B9A5A15 Saigon                     2004   3156.0  
                                              2007    982.0  
AR00MBZ1187B9B5DB1 Memphis Minnie             1930    206.0  
                                              1941    212.0  
                                              1991   1567.0  
                                              1995    166.0  
                                              1997    288.0  
                                              1999    425.0  
                                              2000    496.0  
AR00RBU1187B9B3F1F Matthew Jay                2000    145.0  
                                              2001   1400.0  
AR00TGQ1187B994F29 Paula Toller               2000    661.0  
                                              2007    712.0  
                                              2008    108.0  
AR00Y9I1187B999412 Akercocke                  1999    412.0  
         

In [ ]:
#export and save aggregated data
aggregate_sentiment.to_csv("./sentiment_over_time.csv")

In [ ]:
#also get sentiment information for count and tfidf
combined_count_tfidf_sentiment = lyrics_with_sentiment.copy()[["track_id", "mxm_tid", "word", "count", "is_test", "tf-idf", "summary_sentiment_metric"]]
combined_count_tfidf_sentiment.head()

,track_id,mxm_tid,word,count,is_test,tf-idf,summary_sentiment_metric
0,TRAAAAV128F421A322,4623710,i,6,0,1.867490,0.0
1,TRAAAAV128F421A322,4623710,the,4,0,0.929107,0.0
2,TRAAAAV128F421A322,4623710,you,2,0,0.708444,0.0
3,TRAAAAV128F421A322,4623710,to,2,0,0.563980,0.0
4,TRAAAAV128F421A322,4623710,and,5,0,1.504186,0.0


In [ ]:
combined_count_tfidf_sentiment.to_csv("./combined_count_tfidf_sentiment.csv")

In [ ]:
combined_count_tfidf_sentiment.head(40)

,track_id,mxm_tid,word,count,is_test,tf-idf,summary_sentiment_metric
0,TRAAAAV128F421A322,4623710,i,6,0,1.867490,0.0
1,TRAAAAV128F421A322,4623710,the,4,0,0.929107,0.0
2,TRAAAAV128F421A322,4623710,you,2,0,0.708444,0.0
3,TRAAAAV128F421A322,4623710,to,2,0,0.563980,0.0
4,TRAAAAV128F421A322,4623710,and,5,0,1.504186,0.0
5,TRAAAAV128F421A322,4623710,a,3,0,0.776853,0.0
6,TRAAAAV128F421A322,4623710,me,1,0,0.452941,0.0
7,TRAAAAV128F421A322,4623710,it,1,0,0.463413,0.0
8,TRAAAAV128F421A322,4623710,my,1,0,0.601202,0.0
9,TRAAAAV128F421A322,4623710,is,2,0,0.873126,0.0


In [28]:
from google.colab import files
files.download('./combined_count_tfidf_sentiment.csv')
files.download('./tfidf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

idea: use tf-idf matrix and svd to get latent semantic indexing, top 100 concepts (identified by first word)

 - identify top ~3 concepts per artist per song
 - track concepts of artist over time